In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Exposure Time Calculator\n",
    "\n",
    "In this notebook we will be doing exposure time calculations for point sources, and extended sources with half-light radii of 0.2 arcsec or 0.3 arcsec.\n",
    "\n",
    "When running this notebook you will be asked to select your filter, the Zodiacal light contribution, the S/N, the nature of your source, and the fit applied. \n",
    "\n",
    "The options for each are discussed below:\n",
    "\n",
    "- Filters: F062, F087, F106, F129, F158, F184, F146, F213 \n",
    "- Zodiacal light contributions (multiples of the minimum): 1.2, 1.4, 2.0, 3.5\n",
    "- Source: point sources, objects with a half-light radius (HLR) = 0.2\", objects with a HLR = 0.3\"\n",
    "    - Fit with a PSF (Point source only)\n",
    "    - Fit with a 2 pixel circular aperture (Point source & HLR = 0.2\")\n",
    "    - Fit with a 3 pixel circular aperture \n",
    "    - Fit with a 4 pixel circular aperture\n",
    "    - Fit with a 5 pixel circular aperture (HLR = 0.3\" only)\n",
    "    - Fit with a 6 pixel circular aperture (HLR = 0.2\" & 0.3\" only)\n",
    "    \n",
    "- S/N: 5, 10, 15, 20, 50\n",
    "\n",
    "Exposure times are quantized in multiples of  3 readout frames, with the number of visits/dithers: 1\n",
    "\n",
    "You can calculate either the magnitude for an object at a for a given exposure time and a S/N, or the exposure time needed for a given magnitude."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "from astropy.io import ascii\n",
    "from scipy.interpolate import interp1d\n",
    "import ipywidgets \n",
    "from ipywidgets import widgets"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This notebook works by conducting a simple linear interpolation between the magnitude and exposure time for a given S/N."
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Select you Filter:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "9033127fc37f433ca44a610344d2857a",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Dropdown(description='Filter:', options=('F062', 'F087', 'F106', 'F129', 'F158', 'F184', 'F146', 'F213'), valu…"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "widg_flt = widgets.Dropdown(\n",
    "    options=['F062', 'F087', 'F106', 'F129', 'F158', 'F184', 'F146', 'F213'],\n",
    "    value='F062',\n",
    "    description='Filter:',\n",
    "    disabled=False,)\n",
    "\n",
    "display(widg_flt)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "You selected: F062\n"
     ]
    }
   ],
   "source": [
    "filter = widg_flt.value\n",
    "print(\"You selected: \"+filter)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Select the Zodiacal  light contribution:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "a374a3db736840eb8adbde0fda2ffd44",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Dropdown(description='Zodical light:', options=(1.2, 1.4, 2.0, 3.5), value=1.2)"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "widg_zod = widgets.Dropdown(\n",
    "    options=[1.2, 1.4, 2.0, 3.5],\n",
    "    value=1.2,\n",
    "    description='Zodical light:',\n",
    "    disabled=False,)\n",
    "\n",
    "display(widg_zod)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "You selected: 1.2\n"
     ]
    }
   ],
   "source": [
    "zodi = widg_zod.value\n",
    "print(\"You selected: \"+str(zodi))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Select your kind of source and the fit to it \n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "6621ffb48e3d410885fffdad0100abed",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Dropdown(description='Source:', options=('point source: PSF', 'point source: CircAp = 2', 'point source: CircA…"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "widg_source = widgets.Dropdown(\n",
    "    options=['point source: PSF', 'point source: CircAp = 2', 'point source: CircAp = 3', 'point source: CircAp = 4',\n",
    "            'half-light radius=0.2\": CircAp = 2', 'half-light radius=0.2\": CircAp = 3', 'half-light radius=0.2\": CircAp = 4', 'half-light radius=0.2\": CircAp = 6',\n",
    "            'half-light radius=0.3\": CircAp = 3','half-light radius=0.3\": CircAp = 4', 'half-light radius=0.3\": CircAp = 5', 'half-light radius=0.3\": CircAp = 6'],\n",
    "    value='point source: PSF',\n",
    "    description='Source:',\n",
    "    disabled=False,)\n",
    "\n",
    "display(widg_source)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "You selected: point source: PSF\n"
     ]
    }
   ],
   "source": [
    "source = widg_source.value\n",
    "print(\"You selected: \"+source)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Select the S/N"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "b9c9d353af88458aace947b443e5665a",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "Dropdown(description='S/N:', options=(5, 10, 15, 20, 50), value=5)"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "widg_snr = widgets.Dropdown(\n",
    "    options=[5, 10, 15, 20, 50],\n",
    "    value=5,\n",
    "    description='S/N:',\n",
    "    disabled=False,)\n",
    "\n",
    "display(widg_snr)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "You selected: 5\n"
     ]
    }
   ],
   "source": [
    "snr = widg_snr.value\n",
    "print(\"You selected: \"+str(snr))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Read in the selected data file\n",
    "\n",
    "By selecting a source you are reading in a certain data file. We will do this below using ascii.read."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [],
   "source": [
    "ffn = ['point source: PSF', 'point source: CircAp = 2', 'point source: CircAp = 3', 'point source: CircAp = 4',\n",
    "            'half-light radius=0.2\": CircAp = 2', 'half-light radius=0.2\": CircAp = 3', 'half-light radius=0.2\": CircAp = 4', 'half-light radius=0.2\": CircAp = 6',\n",
    "            'half-light radius=0.3\": CircAp = 3','half-light radius=0.3\": CircAp = 4', 'half-light radius=0.3\": CircAp = 5', 'half-light radius=0.3\": CircAp = 6']\n",
    "ffn2 = ['pts_PSFfit.txt', 'pts_CircAp_2.txt', 'pts_CircAp_3.txt', 'pts_CircAp_4.txt'\n",
    "       'hlr2_CircAp_2.txt', 'hlr2_CircAp_3.txt', 'hlr2_CircAp_4.txt', 'hlr2_CircAp_6.txt',\n",
    "       'hlr3_CircAp_3.txt','hlr3_CircAp_4.txt', 'hlr3_CircAp_5.txt', 'hlr3_CircAp_6.txt']\n",
    "\n",
    "#PS files\n",
    "if source==ffn[0]:\n",
    "    data = ascii.read(ffn2[0])\n",
    "if source==ffn[1]:\n",
    "    data = ascii.read(ffn2[1])\n",
    "if source==ffn[2]:\n",
    "    data = ascii.read(ffn2[2])\n",
    "if source==ffn[3]:\n",
    "    data = ascii.read(ffn2[3])\n",
    "\n",
    "#Half-light radius 0.2\n",
    "if source==ffn[4]:\n",
    "    data = ascii.read(ffn2[4])\n",
    "if source==ffn[5]:\n",
    "    data = ascii.read(ffn2[5])\n",
    "if source==ffn[6]:\n",
    "    data = ascii.read(ffn2[6])\n",
    "if source==ffn[7]:\n",
    "    data = ascii.read(ffn2[7])\n",
    "\n",
    "#Half-light radius 0.3\n",
    "if source==ffn[8]:\n",
    "    data = ascii.read(ffn2[8])\n",
    "if source==ffn[9]:\n",
    "    data = ascii.read(ffn2[9])\n",
    "if source==ffn[10]:\n",
    "    data = ascii.read(ffn2[10])\n",
    "if source==ffn[11]:\n",
    "    data = ascii.read(ffn2[11])\n",
    " "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now we obtain the key values from the file"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [],
   "source": [
    "fl = np.array(data['Filt'], dtype=str)\n",
    "zd = np.array(data['zodi'], dtype=float)\n",
    "sn = np.array(data['SNR'], dtype=float)\n",
    "mag = np.array(data['Mag_AB'], dtype=float)\n",
    "exp = np.array(data['time'], dtype=float)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Next we sort based on what you the user input "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [],
   "source": [
    "#Filter selection\n",
    "a = np.where(fl==filter)\n",
    "fa, za, sa, ma, ea = fl[a], zd[a], sn[a], mag[a], exp[a]\n",
    "\n",
    "#Zodical light selection\n",
    "b = np.where(za==float(zodi))\n",
    "fb, zb, sb, mb, eb = fa[b], za[b], sa[b], ma[b], ea[b]\n",
    "\n",
    "#S/N selection\n",
    "c = np.where(sb==float(snr))\n",
    "fc, zc, sc, mc, ec = fb[c], zb[c], sb[c], mb[c], eb[c]\n",
    "\n",
    "tmin = np.min(ec)\n",
    "tmax = np.max(ec)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Then we set up the interpolation, one based on magnitude and one on exposure time."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [],
   "source": [
    "fm = interp1d(ec, mc) #Return a magnitude\n",
    "ft = interp1d(mc, ec) #Return a time"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# To calculate the magnitude of your object given an exposure time\n",
    "\n",
    "Note that you can only specify times between the minimim and maximum values as given below. If a -9 is returned. this is because the time specified is outside of the given range. "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Minimum allowed time (sec): 10.0\n",
      "Maximum allowed time (sec): 28902.0\n"
     ]
    }
   ],
   "source": [
    "print(\"Minimum allowed time (sec): \"+str(tmin))\n",
    "print(\"Maximum allowed time (sec): \"+str(tmax))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "5072a9eea7eb436fbfe1affac3c4fc5b",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "FloatText(value=10.0, description='Exp time (s):')"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "widg_timein = widgets.FloatText(\n",
    "    value=10,\n",
    "    min= tmin,\n",
    "    max = tmax,\n",
    "    description='Exp time (s):',\n",
    "    disabled=False\n",
    ")\n",
    "\n",
    "display(widg_timein)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Calculated Mag (AB): 20.9\n"
     ]
    }
   ],
   "source": [
    "print(\"Calculated Mag (AB): \"+str(fm(widg_timein.value)))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# To calculate the exposure time needed for your given object\n",
    "\n",
    "Note that you can only specify magnitudes between 20 and 30 AB."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "application/vnd.jupyter.widget-view+json": {
       "model_id": "5a9fc3b5f8584a00942f270bbcefda0b",
       "version_major": 2,
       "version_minor": 0
      },
      "text/plain": [
       "FloatText(value=20.0, description='Exp time (s):')"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "widg_magin = widgets.FloatText(\n",
    "    value=20,\n",
    "    min= 20,\n",
    "    max = 30,\n",
    "    description='Exp time (s):',\n",
    "    disabled=False\n",
    ")\n",
    "\n",
    "display(widg_magin)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Calculated Time (sec): 10.0\n"
     ]
    }
   ],
   "source": [
    "print(\"Calculated Time (sec): \"+str(ft(widg_magin.value)))"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.7"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}